# Code to Track Elapse Time Since a Met Condition
This meant to track the elapsed time since a loss of a signal, useful of signalling comms operations using discrete binary data for digital signals (1/0). This method should be more resistant to inconsistent time intervals due to intepolation of various sensor readings, as compared to comparing signal patterns purely by index number.

Note:
Method 1 [Conditional_ElapsedTime_01()] is faster than method 2[Conditional_ElapsedTime_02()], but they both have the same output results

Credits:
Scripts adapted from: https://stackoverflow.com/questions/53656878/pandas-measure-elapsed-time-since-a-condition

Updated by Randi Ang


In [1]:
# import libraries
import numpy as np
import pandas as pd
import time

In [2]:
# Initialise dataframe for test
# initialize list of lists
data1 = [
        [1657763100000, 1], 
        [1657763100001, 1], 
        [1657763100002, 1],
        [1657763100003, 0],
        [1657763100004, 0], 
        [1657763100005, 0], 
        [1657763100006, 0], 
        [1657763100007, 0], 
        [1657763100008, 0], 
        [1657763100009, 0], 
        [1657763100010, 1], 
        [1657763100011, 1], 
        [1657763100012, 0], 
        [1657763100013, 0], 
        [1657763100014, 0], 
        [1657763100015, 0], 
        [1657763100016, 0], 
        [1657763100017, 0], 
        [1657763100018, 0], 
        [1657763100019, 0],
        [1657763100020, 1] 
       ]
data2 = [
        [1657763100000, 0], 
        [1657763100001, 1], 
        [1657763100002, 1],
        [1657763100003, 0],
        [1657763100004, 0], 
        [1657763100005, 0], 
        [1657763100006, 0], 
        [1657763100007, 0], 
        [1657763100008, 0], 
        [1657763100009, 0], 
        [1657763100010, 1], 
        [1657763100011, 1], 
        [1657763100012, 0], 
        [1657763100013, 0], 
        [1657763100014, 0], 
        [1657763100015, 0], 
        [1657763100016, 0], 
        [1657763100017, 0], 
        [1657763100018, 0], 
        [1657763100019, 0],
        [1657763100020, 1]
       ]
  
# Create the pandas DataFrame
df1 = pd.DataFrame(data1, columns=['Datetime', 'SignalPing'])
df2 = pd.DataFrame(data2, columns=['Datetime', 'SignalPing'])

# Convert time from unix time to human readable datetime
df1['Datetime'] = pd.to_datetime(df1['Datetime'],unit='ms')
df2['Datetime'] = pd.to_datetime(df2['Datetime'],unit='ms')

# Inspect results
print("df1")
print(df1.info())
print(df1.head(10))
print()
print("df2")
print(df2.info())
print(df2.head(10))


df1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Datetime    21 non-null     datetime64[ns]
 1   SignalPing  21 non-null     int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 464.0 bytes
None
                 Datetime  SignalPing
0 2022-07-14 01:45:00.000           1
1 2022-07-14 01:45:00.001           1
2 2022-07-14 01:45:00.002           1
3 2022-07-14 01:45:00.003           0
4 2022-07-14 01:45:00.004           0
5 2022-07-14 01:45:00.005           0
6 2022-07-14 01:45:00.006           0
7 2022-07-14 01:45:00.007           0
8 2022-07-14 01:45:00.008           0
9 2022-07-14 01:45:00.009           0

df2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Datetime 

In [3]:
def Conditional_ElapsedTime_01(CondMonitoringCol, TargetCondition, DatetimeCol, TimeResolution = "timedelta64[ms]", IntegerOutput = True):
    # Argument Input Variable format    
    # CondMonitoringCol = df1["SignalPing"] --> points to dataframe column to monitor for target condition
    # TargetCondition = anything --> defines target condition where elapse time resets from
    # DatetimeCol = df1["Datetime"] --> point to dataframe column with datetime values
    # TimeResolution = "timedelta64[s]" | "timedelta64[ms]" | "timedelta64[us]" | "timedelta64[ns]"
    # IntegerOutput = True | False (output in datetime difference format)
    
    # Get elapse time (numpy method)
    # on numpy unique()
    # u = the indices of the input array that give the unique values
    # f = the indices of the unique array that reconstruct the input array
    # i = the number of times each unique value comes up in the input array
    # "df1["SignalPing"].eq(1).values.cumsum()" gets last index where df1["SignalPing"] = 1
    # Note: The counting only begins after the first target condition detected
    u, f, i = np.unique(CondMonitoringCol.eq(TargetCondition).values.cumsum(), return_index = True, return_inverse = True)
    t = DatetimeCol.values
    OutputCol = t - t[f[i]]
    
    if (IntegerOutput == True):
        # Convert elapse time into an integer
        # Define time resolution using "timedelta64[ms]"
        # Currently set for milliseconds (ms) resolution
        # use s, us, ns for seconds, microseconds and nanoseconds resolution respectively
        OutputCol = (t - t[f[i]]).astype(TimeResolution).astype(int)
    else:
        pass
    
    return OutputCol

In [4]:
# get the start time
st = time.time()

# Compute Elapse Time
df1['Elapsed Time 01 - ms'] = Conditional_ElapsedTime_01(df1["SignalPing"], 1, df1["Datetime"], TimeResolution = "timedelta64[ms]", IntegerOutput = True)
df2['Elapsed Time 01 - ms'] = Conditional_ElapsedTime_01(df2["SignalPing"], 1, df2["Datetime"], TimeResolution = "timedelta64[ms]", IntegerOutput = True)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

# Inspect results
print("df1")
print(df1.info())
print(df1)
print()
print("df2")
print(df2.info())
print(df2)

Execution time: 0.001999378204345703 seconds
df1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Datetime              21 non-null     datetime64[ns]
 1   SignalPing            21 non-null     int64         
 2   Elapsed Time 01 - ms  21 non-null     int32         
dtypes: datetime64[ns](1), int32(1), int64(1)
memory usage: 548.0 bytes
None
                  Datetime  SignalPing  Elapsed Time 01 - ms
0  2022-07-14 01:45:00.000           1                     0
1  2022-07-14 01:45:00.001           1                     0
2  2022-07-14 01:45:00.002           1                     0
3  2022-07-14 01:45:00.003           0                     1
4  2022-07-14 01:45:00.004           0                     2
5  2022-07-14 01:45:00.005           0                     3
6  2022-07-14 01:45:00.006           0                 

In [5]:
def Conditional_ElapsedTime_02(CondMonitoringCol, TargetCondition, DatetimeCol, TimeResolution = "s", IntegerOutput = True):
    # Argument Input Variable format    
    # CondMonitoringCol = df1["SignalPing"] --> points to dataframe column to monitor for target condition
    # TargetCondition = anything --> defines target condition where elapse time resets from
    # DatetimeCol = df1["Datetime"] --> point to dataframe column with datetime values
    # TimeResolution = "s" | "ms" | "us" | "ns"
    # IntegerOutput = True | False (output in datetime difference format)
    
    # Trigger workflow based on time resolution of input time
    # Output values (integer format) would be in the same resolution
    if TimeResolution == "s":
        TimeAdj = 10**9
    elif TimeResolution == "ms":
        TimeAdj = 10**6
    elif TimeResolution == "us":
        TimeAdj = 10**3
    elif TimeResolution == "ns":
        TimeAdj = 1
    
    # Convert datetime to unix time for computation
    UnixTime = DatetimeCol.astype(np.int64) / TimeAdj
    
    # Compute Elapsed Time
    OutputCol = (UnixTime - UnixTime.groupby(CondMonitoringCol.eq(TargetCondition).cumsum()).transform('first')).round(0).astype(np.int64)
    
    
    if (IntegerOutput == True):
        pass
    else:
        # Convert elapse time into time delta
        OutputCol = pd.to_timedelta(OutputCol, unit = TimeResolution)
    
    return OutputCol

In [6]:
# get the start time
st = time.time()

df1['Elapsed Time 02 - ms'] = Conditional_ElapsedTime_02(df1["SignalPing"], 1, df1["Datetime"], TimeResolution = "ms", IntegerOutput = True)
df2['Elapsed Time 02 - ms'] = Conditional_ElapsedTime_02(df2["SignalPing"], 1, df2["Datetime"], TimeResolution = "ms", IntegerOutput = True)

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')

# Inspect results
print("df1")
print(df1.info())
print(df1)
print()
print("df2")
print(df2.info())
print(df2)

Execution time: 0.0029997825622558594 seconds
df1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Datetime              21 non-null     datetime64[ns]
 1   SignalPing            21 non-null     int64         
 2   Elapsed Time 01 - ms  21 non-null     int32         
 3   Elapsed Time 02 - ms  21 non-null     int64         
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage: 716.0 bytes
None
                  Datetime  SignalPing  Elapsed Time 01 - ms  \
0  2022-07-14 01:45:00.000           1                     0   
1  2022-07-14 01:45:00.001           1                     0   
2  2022-07-14 01:45:00.002           1                     0   
3  2022-07-14 01:45:00.003           0                     1   
4  2022-07-14 01:45:00.004           0                     2   
5  2022-07-14 01:45:00.005           0 

In [7]:
# Inspect results
df1

,Datetime,SignalPing,Elapsed Time 01 - ms,Elapsed Time 02 - ms
0,2022-07-14 01:45:00.000,1,0,0
1,2022-07-14 01:45:00.001,1,0,0
2,2022-07-14 01:45:00.002,1,0,0
3,2022-07-14 01:45:00.003,0,1,1
4,2022-07-14 01:45:00.004,0,2,2
5,2022-07-14 01:45:00.005,0,3,3
6,2022-07-14 01:45:00.006,0,4,4
7,2022-07-14 01:45:00.007,0,5,5
8,2022-07-14 01:45:00.008,0,6,6
9,2022-07-14 01:45:00.009,0,7,7


In [8]:
# Inspect results
df2

,Datetime,SignalPing,Elapsed Time 01 - ms,Elapsed Time 02 - ms
0,2022-07-14 01:45:00.000,0,0,0
1,2022-07-14 01:45:00.001,1,0,0
2,2022-07-14 01:45:00.002,1,0,0
3,2022-07-14 01:45:00.003,0,1,1
4,2022-07-14 01:45:00.004,0,2,2
5,2022-07-14 01:45:00.005,0,3,3
6,2022-07-14 01:45:00.006,0,4,4
7,2022-07-14 01:45:00.007,0,5,5
8,2022-07-14 01:45:00.008,0,6,6
9,2022-07-14 01:45:00.009,0,7,7
